Currently, each variable has 19 features.

Here, we will add additional global feature(s) by appending them to each variable node's features.

In [ ]:
%load_ext autoreload
%autoreload

from retro_branching.environments import EcoleBranching
from retro_branching.agents import PseudocostBranchingAgent

import ecole
import numpy as np

In [ ]:
class NodeBipartiteWithGlobalFeatures(ecole.observation.NodeBipartite):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def before_reset(self, model):
        super().before_reset(model)
        
        self.init_dual_bound = None
        self.init_primal_bound = None
        
        
    def extract(self, model, done):
        # get the NodeBipartite obs
        obs = super().extract(model, done)
        
        # label vars in obs with global feats
        print(f'Ecole obs column feats:\n{obs.column_features.shape} {obs.column_features[0]}')
        
        m = model.as_pyscipopt()
        
        if self.init_dual_bound is None:
            self.init_dual_bound = m.getDualbound()
            self.init_primal_bound = m.getPrimalbound()
            self.init_lower_bound = m.getCurrentNode().getLowerbound()
            self.init_best_feasible_estimate = m.getCurrentNode().getEstimate()
            
            print(f'init_dual_bound: {self.init_dual_bound}')
            print(f'init_primal_bound: {self.init_primal_bound}')
            print(f'init_lower_bound: {self.init_lower_bound}')
            print(f'init_best_feasible_estimate: {self.init_best_feasible_estimate}')
            
        # dual/primal bound features
#         dual_bound_frac_change = self.init_dual_bound / m.getDualbound()
#         primal_bound_frac_change = self.init_primal_bound / m.getPrimalbound()
#         print(f'debug: {dual_bound_frac_change} {primal_bound_frac_change}')
#         dual_bound_frac_change = abs(1-(min(self.init_dual_bound, m.getDualbound()) / max(self.init_dual_bound, m.getDualbound())))
#         primal_bound_frac_change = abs(1-(min(self.init_primal_bound, m.getPrimalbound()) / max(self.init_primal_bound, m.getPrimalbound())))
        dual_bound_frac_change = abs(self.init_dual_bound - m.getDualbound()) / self.init_dual_bound
        primal_bound_frac_change = abs(self.init_primal_bound - m.getPrimalbound()) / self.init_primal_bound
        curr_primal_dual_bound_gap_frac = m.getGap()
        
        # global tree features
        num_leaves_frac = m.getNLeaves() / m.getNNodes()
        num_feasible_leaves_frac = m.getNFeasibleLeaves() / m.getNNodes()
        num_infeasible_leaves_frac = m.getNInfeasibleLeaves() / m.getNNodes()
        # getNSolsFound() raises attribute error for some reason. Not supported by Ecole?
#         num_feasible_sols_found_frac = m.getNSolsFound() / m.getNNodes() # gives idea for how hard problem is, since harder problems may have more sparse feasible solutions?
#         num_feasible_best_sols_found_frac = m.getNBestSolsFound() / m.getNSolsFound()
        num_lp_iterations_frac = m.getNNodes() / m.getNLPIterations()
        
        # focus node features
#         num_children_frac = m.getNChildren() / m.getNNodes()
        num_siblings_frac = m.getNSiblings() / m.getNNodes()





        # NEW FEATURES
        
        # 1. is current node the best node?
        curr_node = m.getCurrentNode()
        best_node = m.getBestNode()
#         print(f'best child: {m.getBestChild()}')
#         print(f'best sibling: {m.getBestSibling()}')
        print(f'\n\ncurr node: {curr_node.getNumber()}')
        if best_node is not None:
            print(f'best node: {best_node.getNumber()}')
            if curr_node.getNumber() == best_node.getNumber():
                is_curr_node_best = 1
            else:
                is_curr_node_best = 0
        else:
            # no best node found yet
            print(f'best node: {best_node}')
            is_curr_node_best = 0
            
        # 2. is current node's parent the best node?
        parent_node = curr_node.getParent()
        if parent_node is not None:
            print(f'parent node: {parent_node.getNumber()}')
        else:
            print(f'parent node: {parent_node}')
        if parent_node is not None and best_node is not None:
            if parent_node.getNumber() == best_node.getNumber():
                is_curr_node_parent_best = 1
            else:
                is_curr_node_parent_best = 0
        else:
            # node has no parent node or no best node found yet
            is_curr_node_parent_best = 0
        
        # 3. current node depth
        curr_node_depth = m.getDepth() / m.getNNodes()
        
        # 4. node lower bound relative to initial dual bound
        curr_node_lower_bound_relative_to_init_dual_bound = self.init_dual_bound / curr_node.getLowerbound()
        
        # 5. node lower bound relative to current global best dual bound
        curr_node_lower_bound_relative_to_curr_dual_bound =  m.getDualbound() / curr_node.getLowerbound()
        
        # 6/7/8. curr node number of bound changes due to 1. branching 2. constraint propagation 3. propagation
        num_branching_changes, num_constraint_prop_changes, num_prop_changes = curr_node.getNDomchg()
        total_num_changes = num_branching_changes + num_constraint_prop_changes + num_prop_changes
        try:
            branching_changes_frac = num_branching_changes / total_num_changes
        except ZeroDivisionError:
            branching_changes_frac = 0
        try:
            constraint_prop_changes_frac = num_constraint_prop_changes / total_num_changes
        except ZeroDivisionError:
            constraint_prop_changes_frac = 0
        try:
            prop_changes_frac = num_prop_changes / total_num_changes
        except ZeroDivisionError:
            prop_changes_frac = 0
        
        # 9. num parent branchings which led to creating this node
        parent_branching_changes_frac = curr_node.getNParentBranchings() / m.getNNodes()
        
        # 10/11. num feasible solutions found so far
        num_feasible_solutions_relative_to_nodes = m.getNNodes() / m.getNNodes()
        num_feasible_solutions_relative_to_lps = m.getNSols() / m.getNLPs()
        
#         # 11. is best node None?
#         if best_node is None:
#             is_best_node_none = 1
#         else:
#             is_best_node_none = 0
            
        # 12/13. is best sibling of current focus node none, and is it the best node?
        best_sibling = m.getBestSibling()
        if best_sibling is None:
            is_best_sibling_none = 1
            is_best_sibling_best_node = 0
        else:
            is_best_sibling_none = 0
            if best_node is not None:
                if best_sibling.getNumber() == best_node.getNumber():
                    is_best_sibling_best_node = 1
                else:
                    is_best_sibling_best_node = 0
            else:
                is_best_sibling_best_node = 0
                
        # 14/15/16. best sibling lower bound
        if best_sibling is not None:
            best_sibling_lower_bound_relative_to_init_dual_bound = self.init_dual_bound / best_sibling.getLowerbound()
            best_sibling_lower_bound_relative_to_curr_dual_bound = m.getDualbound() / best_sibling.getLowerbound()
            best_sibling_lower_bound_relative_to_curr_node_lower_bound = best_sibling.getLowerbound() / curr_node.getLowerbound()
        else:
            best_sibling_lower_bound_relative_to_init_dual_bound = 0
            best_sibling_lower_bound_relative_to_curr_dual_bound = 0
            best_sibling_lower_bound_relative_to_curr_node_lower_bound = 0
            
            
        # max change in primal and dual bound frac
        primal_dual_gap = abs(m.getPrimalbound() - m.getDualbound())
        max_dual_bound_frac_change = primal_dual_gap / self.init_dual_bound
        max_primal_bound_frac_change = primal_dual_gap / self.init_primal_bound
            
            
            
        
        
        print('\n> dual/primal bound features <')
        print(f'dual_bound_frac_change: {dual_bound_frac_change}')
        print(f'primal_bound_frac_change: {primal_bound_frac_change}')
        print(f'curr_primal_dual_bound_gap_frac: {curr_primal_dual_bound_gap_frac}')
        print(f'max_dual_bound_frac_change {max_dual_bound_frac_change}')
        print(f'max_primal_bound_frac_change: {max_primal_bound_frac_change}')
        
        print('\n> global tree features <')
        print(f'num_leaves_frac: {num_leaves_frac}')
        print(f'num_feasible_leaves_frac: {num_feasible_leaves_frac}')
        print(f'num_infeasible_leaves_frac: {num_infeasible_leaves_frac}')
#         print(f'num_feasible_sols_found_frac: {num_feasible_sols_found_frac}')
#         print(f'num_feasible_best_sols_found_frac: {num_feasible_best_sols_found_frac}')
        print(f'num_lp_iterations_frac: {num_lp_iterations_frac}')
        print(f'num_feasible_solutions_frac: {num_feasible_solutions_relative_to_nodes}')
        print(f'num_feasible_solutions_relative_to_lps: {num_feasible_solutions_relative_to_lps}')
#         print(f'is_best_node_none: {is_best_node_none}')
        
        print('\n> focus node features <')
#         print(f'num_children_frac: {num_children_frac}')
        print(f'num_siblings_frac: {num_siblings_frac}')
        print(f'is_curr_node_best: {is_curr_node_best}')
        print(f'is_curr_node_parent_best: {is_curr_node_parent_best}')
        print(f'curr_node_depth: {curr_node_depth}')
        print(f'curr_node_lower_bound_relative_to_init_dual_bound: {curr_node_lower_bound_relative_to_init_dual_bound}')
        print(f'curr_node_lower_bound_relative_to_curr_dual_bound: {curr_node_lower_bound_relative_to_curr_dual_bound}')
        print(f'branching_changes_frac: {branching_changes_frac}')
        print(f'constraint_prop_changes_frac: {constraint_prop_changes_frac}')
        print(f'prop_changes_frac: {prop_changes_frac}')
        print(f'parent_branching_changes_frac: {parent_branching_changes_frac}')
        print(f'is_best_sibling_none: {is_best_sibling_none}')
        print(f'is_best_sibling_best_node: {is_best_sibling_best_node}')
        print(f'best_sibling_lower_bound_relative_to_init_dual_bound: {best_sibling_lower_bound_relative_to_init_dual_bound}')
        print(f'best_sibling_lower_bound_relative_to_curr_dual_bound: {best_sibling_lower_bound_relative_to_curr_dual_bound}')
        print(f'best_sibling_lower_bound_relative_to_curr_node_lower_bound: {best_sibling_lower_bound_relative_to_curr_node_lower_bound}')
        
        


        
        # add feats to each variable
        feats_to_add = np.array([[dual_bound_frac_change,
                                 primal_bound_frac_change,
                                 curr_primal_dual_bound_gap_frac,
                                 num_leaves_frac,
                                 num_feasible_leaves_frac,
                                 num_infeasible_leaves_frac,
                                 num_lp_iterations_frac,
                                 num_feasible_solutions_relative_to_nodes,
                                 num_feasible_solutions_relative_to_lps,
                                 num_siblings_frac,
                                 is_curr_node_best,
                                 is_curr_node_parent_best,
                                 curr_node_depth,
                                 curr_node_lower_bound_relative_to_init_dual_bound,
                                 curr_node_lower_bound_relative_to_curr_dual_bound,
                                 branching_changes_frac,
                                 constraint_prop_changes_frac,
                                 prop_changes_frac,
                                 parent_branching_changes_frac,
                                 num_feasible_solutions_relative_to_nodes,
                                 num_feasible_solutions_relative_to_lps,
                                 is_best_sibling_none,
                                 is_best_sibling_best_node,
                                 best_sibling_lower_bound_relative_to_init_dual_bound,
                                 best_sibling_lower_bound_relative_to_curr_dual_bound,
                                 best_sibling_lower_bound_relative_to_curr_node_lower_bound] for _ in range(obs.column_features.shape[0])])
        
        obs.column_features = np.column_stack((obs.column_features, feats_to_add))
        
        print(f'Updated obs column feats:\n{obs.column_features.shape} {obs.column_features[0]}')
                
        return obs

In [ ]:
env = EcoleBranching(observation_function=NodeBipartiteWithGlobalFeatures())

# instances
instances = ecole.instance.SetCoverGenerator(n_rows=100, n_cols=100, density=0.05)
instances = ecole.instance.SetCoverGenerator(n_rows=500, n_cols=1000, density=0.05)

In [ ]:
obs = None
while obs is None:
    env.seed(0)
    instance = next(instances)
#     instance_before_reset = instance.copy_orig()
    obs, action_set, reward, done, info = env.reset(instance)

Lets try to solve an instance and see how our observation values change (should all stay around 1.0 to be same magnitude as other features)

In [ ]:
agent = PseudocostBranchingAgent()

obs = None
while obs is None:
    env.seed(0)
    instance = next(instances)
    instance_before_reset = instance.copy_orig()
    obs, action_set, reward, done, info = env.reset(instance)
total_return = 0
while not done:
    action, _ = agent.action_select(action_set, env.model, done)
    obs, action_set, reward, done, info = env.step(action)
    total_return += reward['dual_bound_frac']
    print(f'dual_bound_frac reward: {reward["dual_bound_frac"]} | total: {total_return}')